In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
df = data.train_df('small')

In [4]:
df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1.0


In [5]:
# cast the impressions and the prices to lists
clickout_rows = df[df.prices.notnull()][['impressions','prices']]

clickout_rows['impressions'] = clickout_rows.impressions.str.split('|') #.apply(lambda x: list(map(int,x)))
clickout_rows['prices'] = clickout_rows.prices.str.split('|') #.apply(lambda x: list(map(int,x)))

In [6]:
clickout_rows.head()

,impressions,prices
13,"[3400638, 1253714, 3367857, 5100540, 1088584, 666916, 54833, 2922310, 9711560, 109038, 666856, 10077318, 1431482, 129343, 6339822, 6806806, 1041528, 109013, 3909420, 55088, 3095758, 109018, 54885, 1257342, 2595006]","[95, 66, 501, 112, 95, 100, 101, 72, 82, 56, 56, 143, 70, 25, 71, 162, 73, 143, 188, 118, 77, 131, 143, 49, 165]"
15,"[55109, 129343, 54824, 2297972, 109014, 1257342, 1031578, 109018, 1332971, 666916, 54833, 54885, 2237222, 10077318, 1166793, 9132132, 1474297, 3909420, 6622154, 55091, 8444418, 54804, 2050977, 666936, 55088]","[162, 25, 150, 143, 101, 49, 118, 131, 18, 100, 101, 143, 51, 143, 123, 36, 66, 188, 124, 138, 94, 137, 180, 112, 118]"
115,"[2795374, 5582964, 1088390, 2781070, 1258068, 1271962, 3184892, 148884, 3528776, 107183, 5156744, 107048, 1907333, 3370484, 6003326, 8436316, 9025316, 125181, 3861490, 131257, 4415954, 107162, 3143352, 6652864, 8118684]","[64, 54, 36, 121, 76, 81, 92, 40, 73, 52, 98, 104, 56, 414, 67, 111, 21, 122, 55, 104, 56, 64, 40, 29, 44]"
121,"[12693, 46363, 81657, 18448, 47687, 152913, 18417, 927627, 4147572, 819616, 18380, 47333, 18425, 1255377, 4043482, 1032816, 18388, 1216900, 106471, 106567, 12667, 84082, 1152188, 98737, 47410]","[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, 107, 86, 98, 83, 98, 103, 105, 99, 106, 99, 98, 96, 96, 105, 99]"
122,"[12693, 46363, 81657, 18448, 47687, 152913, 18417, 927627, 4147572, 819616, 18380, 47333, 18425, 1255377, 4043482, 1032816, 18388, 1216900, 106471, 106567, 12667, 84082, 1152188, 98737, 47410]","[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, 107, 86, 98, 83, 98, 103, 105, 99, 106, 99, 98, 96, 96, 105, 99]"


In [7]:
clickout_rows = pd.DataFrame({col:np.concatenate(clickout_rows[col].values) for col in clickout_rows.columns }).astype('int')

In [8]:
clickout_rows.head()

,impressions,prices
0,3400638,95
1,1253714,66
2,3367857,501
3,5100540,112
4,1088584,95


In [11]:
res_df = clickout_rows.groupby('impressions').agg(['mean','std']).reset_index()
res_df.columns = ['_'.join(x) for x in res_df.columns.ravel()]
res_df = res_df.rename(columns={'impressions_': 'item_id'})
res_df['prices_std'] = res_df['prices_std'].fillna(0)
res_df

,reference,prices_mean,prices_std
0,5001,87.818182,5.963525
1,5002,94.416667,14.506790
2,5003,106.230769,0.832050
3,5004,146.181818,15.012116
4,5005,160.000000,0.000000
5,5006,220.000000,0.000000
6,5009,220.000000,0.000000
7,5011,104.000000,0.000000
8,5013,135.000000,0.000000
9,5014,215.315789,218.159232


In [4]:
from extract_features.rnn.impressions_average_price import 
f = 

In [21]:
clickout_rows.dtypes

impressions    int64
prices         int64
dtype: object

In [33]:
res_df = clickout_rows.groupby('impressions').agg(['mean','count','std',list]).reset_index()

In [5]:
f.join_to(df, one_hot=True)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_price_position_in_last_clickout feature read
reference_price_position_in_last_clickout onehot completed


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price_pos_0,price_pos_1,price_pos_2,price_pos_3,price_pos_4,price_pos_5,price_pos_6,price_pos_7,price_pos_8,price_pos_9,price_pos_10,price_pos_11,price_pos_12,price_pos_13,price_pos_14,price_pos_15,price_pos_16,price_pos_17,price_pos_18,price_pos_19,price_pos_20,price_pos_21,price_pos_22,price_pos_23,price_pos_24
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
res

['prices_mean', 'prices_count', 'prices_std', 'prices_list']

In [43]:
res_df.columns = ["_".join(x) for x in res_df.columns.ravel()]
res_df

,prices_mean,prices_count,prices_std,prices_list
impressions,,,,
5001,87.818182,11,5.963525,"[82, 84, 94, 94, 94, 94, 94, 82, 82, 82, 84]"
5002,94.416667,12,14.506790,"[105, 105, 105, 105, 105, 105, 105, 91, 91, 72, 72, 72]"
5003,106.230769,13,0.832050,"[106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 109]"
5004,146.181818,11,15.012116,"[139, 139, 139, 139, 139, 139, 139, 169, 169, 169, 128]"
5005,160.000000,1,NaN,[160]
5006,220.000000,1,NaN,[220]
5009,220.000000,2,0.000000,"[220, 220]"
5011,104.000000,2,0.000000,"[104, 104]"
5013,135.000000,4,0.000000,"[135, 135, 135, 135]"
